In [ ]:


#Установка PySpark
!pip install pyspark

#Импорт библиотек
import random
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
from datetime import datetime, timedelta
import shutil  # Для переименования файла
import os     # Для работы с файловой системой

#Создание Spark-сессии
spark = SparkSession.builder.master("local[*]").appName("synthetic_data").getOrCreate()

#Параметры и список продуктов
products = ['Наушники', 'Мышь', 'Клавиатура', 'Монитор', 'Коврик']

num_records = 1000  # Количество строк (можно менять)

start_date = datetime.now() - timedelta(days=365)

#Функция для генерации синтетических данных
def generate_data(num_records):
    data = []
    for _ in range(num_records):
        date = start_date + timedelta(days=random.randint(0, 365))
        user_id = random.randint(1000, 9999)
        product = random.choice(products)
        quantity = random.randint(1, 10)
        price = round(random.uniform(10, 5000), 2)
        data.append((date.strftime('%Y-%m-%d'), user_id, product, quantity, price))
    return data

#Генерация данных
data = generate_data(num_records)

#Определение схемы для DataFrame
schema = StructType([
    StructField("Дата", StringType(), True),
    StructField("UserID", IntegerType(), True),
    StructField("Продукт", StringType(), True),
    StructField("Количество", IntegerType(), True),
    StructField("Цена", FloatType(), True)
])

#Создание DataFrame
df = spark.createDataFrame(data, schema)

#Указание имени файла для сохранения
filename = 'DataFrame.csv'  # Задай своё имя файла
output_dir = '/content/3.6/'  # Папка для сохранения

#Сохранение данных в один CSV файл
df.coalesce(1).write.mode("overwrite").csv(f'{output_dir}/temp_csv', header=True)

#Переименование единственного файла внутри сгенерированной папки
temp_csv_path = f'{output_dir}/temp_csv'
for file in os.listdir(temp_csv_path):
    if file.endswith(".csv"):
        shutil.move(f'{temp_csv_path}/{file}', f'{output_dir}/{filename}')
        break

#Удаление временной папки
shutil.rmtree(temp_csv_path)